In [107]:
from geopy.geocoders import Nominatim
import requests 
import pandas as pd
import numpy as np

from pandas.io.json import json_normalize 

In [239]:
CLIENT_ID = "DPR3RTKJ5T3ICFZ0AWISZHKMN235MKE2PF1YRN0VD3R3CTRK"
CLIENT_SECRET = "TRIO3ABTD1KQSETYKNQVGNJCNZGRCUOC4MBPXGUN5ZWCJCNO"
VERSION = "20170513"
LIMIT = 1000

In [240]:
address = "315 Hudson Street New York, NY"

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude 
longitude = location.longitude 

search_query = "hospitals"

In [261]:
radius = 10000
category_id = "4bf58dd8d48988d104941735"
url="https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


In [260]:
results = requests.get(url).json()

In [255]:
hospitals = results["response"]["venues"]

dataframe = json_normalize(hospitals)
dataframe.head()

,allowMenuUrlEdit,beenHere.lastCheckinExpiredAt,categories,contact.facebook,contact.facebookName,contact.facebookUsername,contact.formattedPhone,contact.phone,contact.twitter,hasPerk,...,specials.count,specials.items,stats.checkinsCount,stats.tipCount,stats.usersCount,storeId,url,venueChains,venueRatingBlacklisted,verified
0,True,0,"[{u'pluralName': u'Medical Centers', u'primary...",130828181135,New York City Health and Hospitals Corporation,nychhc,(212) 442-3853,2124423853,hhcnyc,False,...,0,[],30,0,9,NaN,http://t.co/uqT8kyqduH,[{u'id': u'5569faf8a7c8896abe7d5323'}],True,True
1,NaN,0,[{u'pluralName': u'Professional & Other Places...,NaN,NaN,NaN,NaN,NaN,NaN,False,...,0,[],29,0,6,NaN,NaN,[],True,False
2,NaN,0,"[{u'pluralName': u'Hospitals', u'primary': Tru...",103256838688,New York University,NYU,(212) 263-7300,2122637300,nyuniversity,False,...,0,[],39973,99,8667,NaN,http://www.nyu.edu,[{u'id': u'556c9290a7c8a11a36bd78ac'}],True,True
3,NaN,0,"[{u'pluralName': u'City Halls', u'primary': Tr...",130828181135,New York City Health and Hospitals Corporation,nychhc,NaN,NaN,hhcnyc,False,...,0,[],297,1,48,NaN,http://t.co/uqT8kyqduH,[{u'id': u'5569faf8a7c8896abe7d5323'}],True,True
4,NaN,0,"[{u'pluralName': u'Government Buildings', u'pr...",NaN,NaN,NaN,NaN,NaN,NaN,False,...,0,[],12,0,7,NaN,NaN,[],True,False


In [256]:
filtered_columns = ['name', 'url', 'categories'] + [col for col in dataframe.columns if col.startswith('location')] + ['id']
dataframe_filtered = dataframe.ix[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row["categories"]
    except:
        categories_list = row["venue.categories"]
    
    if len(categories_list) == 0:
         return None 
    else:
        return categories_list[0]["name"].encode('ascii', errors="ignore")
    
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split(".")[-1] for column in dataframe_filtered.columns]
dataframe_filtered

        

,name,url,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,HHC Health and Hospitals,http://t.co/uqT8kyqduH,Medical Center,346 Broadway #5,US,NaN,United States,NaN,1095,"[346 Broadway #5, NY 10013, United States]","[{u'lat': 40.7188246952, u'lng': -73.999332979...",40.718825,-73.999333,10013,New York,4d932b2362ad5481124d7b4b
1,NYC Health and Hospitals Corp.,NaN,Professional & Other Places,361-363 Broadway,US,New York,United States,NaN,1003,"[361-363 Broadway, New York, NY 10013, United ...","[{u'lat': 40.7174638615, u'lng': -74.005943798...",40.717464,-74.005944,10013,NY,4dadfb50a86e29d01a3ae715
2,NYU Langone Medical Center,http://www.nyu.edu,Hospital,550 1st Ave,US,New York,United States,btwn E 30th & E 34th St,3317,"[550 1st Ave (btwn E 30th & E 34th St), New Yo...",NaN,40.742285,-73.974374,10016,NY,4a36c1b9f964a520dd9d1fe3
3,NYC Health & Hospitals Corporation,http://t.co/uqT8kyqduH,City Hall,NaN,US,New York,United States,NaN,1234,"[New York, NY 10013, United States]","[{u'lat': 40.7159296944, u'lng': -74.002776143...",40.715930,-74.002776,10013,NY,4eef65210aaf1d45b417dc4f
4,"NYC Department Of Health, Hospitals and Sanita...",NaN,Government Building,125 Worth St,US,New York,United States,NaN,1660,"[125 Worth St, New York, NY 10013, United States]","[{u'lat': 40.7133962345, u'lng': -73.997955696...",40.713396,-73.997956,10013,NY,4f0dc0a0e4b0254b4d12ed73
5,NYC Health and Hospitals Corporation,NaN,Office,160 Water St,US,New York,United States,NaN,2203,"[160 Water St, New York, NY 10038, United States]","[{u'lat': 40.706656222, u'lng': -74.0055421062...",40.706656,-74.005542,10038,NY,4fb3bd9ee4b049bc6b518153
6,NYU Hospitals Center,NaN,Medical School,560 1st Ave,US,New York,United States,NaN,3259,"[560 1st Ave, New York, NY 10016, United States]","[{u'lat': 40.7418911438, u'lng': -73.974856044...",40.741891,-73.974856,10016,NY,527c2d51498e2da8e2b17dcf
7,Gouverneur Health,http://t.co/uqT8kyqduH,Hospital,227 Madison St,US,New York,United States,between Clinton Street & Jefferson Street,2230,[227 Madison St (between Clinton Street & Jeff...,"[{u'lat': 40.7129320447, u'lng': -73.988044708...",40.712932,-73.988045,10002,NY,4b773c05f964a520548a2ee3
8,NYU Langone Hospitals Center Pharmacy,NaN,Hospital,530 1st Ave,US,New York,United States,East 31Street,3271,"[530 1st Ave (East 31Street), New York, NY 100...","[{u'lat': 40.7419085511, u'lng': -73.97470682,...",40.741909,-73.974707,10016,NY,4ff84741e4b0b8fdaa7e9aad
9,NYC Department of Health & Mental Hygiene,http://www.nyc.gov/html/doh/html/home/home.shtml,Government Building,125 Worth St,US,New York,United States,at Leonard St.,1260,"[125 Worth St (at Leonard St.), New York, NY 1...","[{u'lat': 40.71567902, u'lng': -74.0027933587,...",40.715679,-74.002793,10013,NY,4c22388b99282d7f8bbf66b0


In [258]:
import pixiedust 


In [259]:
display(dataframe_filtered)